In [192]:
import pandas as pd
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import  mean_squared_error, r2_score, make_scorer
import numpy as np
import joblib
import re



In [193]:
df = pd.read_csv("cars_for_sale(uncleaned).csv")
df

,Car,Condition,Mileage,Price,Basics Info,Vehicle History Info,Vehicle Reviews Info,Seller Rating,Seller Rating Count,Seller Address
0,2024 Lexus LC 500 Base,New,0 mi.,"$112,865MSRP $118,865","{'Exterior color': ' Caviar ', 'Interior color...",{},{},4.7,"(1,261 reviews)","1250 W Division St Chicago, IL 60642"
1,2007 Acura TSX Base,Used,"61,110 mi.","$11,295",{'Exterior color': ' Alabaster Silver Metallic...,{'Accidents or damage': 'At least 1 accident o...,{},4.2,(440 reviews),"1301 N Elston Ave Chicago, IL 60642"
2,2016 McLaren 675LT Base,Used,"6,305 mi.","$219,997$5,464 price drop","{'Exterior color': ' McLaren Orange ', 'Interi...",{'Accidents or damage': 'At least 1 accident o...,"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",3.1,(421 reviews),"1561 N Fremont St Chicago, IL 60642"
3,2016 Audi TTS 2.0T quattro,Used,"65,715 mi.","$23,999","{'Exterior color': ' Black ', 'Interior color'...",{'Accidents or damage': 'At least 1 accident o...,"{'Comfort': '3.0', 'Interior': '5.0', 'Perform...",3.6,(123 reviews),"560 E North Ave West Chicago, IL 60185"
4,2018 BMW 740e xDrive iPerformance,Used,"19,830 mi.","$39,799$100 price drop","{'Exterior color': ' Imperial Blue Metallic ',...","{'Accidents or damage': 'None reported', 'Clea...","{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.4,(91 reviews),"6539 Ogden Ave Berwyn, IL 60402"
...,...,...,...,...,...,...,...,...,...,...
9241,NaN,NaN,NaN,NaN,{},{},{},NaN,NaN,NaN
9242,2022 BMW X3 xDrive30i,Used,"48,804 mi.","$27,979$998 price drop","{'Exterior color': ' Dark Graphite Metallic ',...","{'Accidents or damage': 'None reported', '1-ow...",{},4.8,"(3,739 reviews)","1313 Rand Road Des Plaines, IL 60016"
9243,2024 GMC Sierra 1500 Pro,New,3 mi.,"$51,080MSRP $51,080","{'Exterior color': ' Summit White ', 'Interior...",{},"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.5,"(3,896 reviews)","8425 159th St Tinley Park, IL 60487"
9244,2012 GMC Terrain SLT-1,Used,"146,694 mi.","$8,995","{'Exterior color': ' Black ', 'Interior color'...",{'Accidents or damage': 'At least 1 accident o...,{},NaN,NaN,"13840 South Pulaski Road Crestwood, IL 60445"


In [194]:
def clean_price(price):
    if isinstance(price,str):
        price = price.split()[0]
        price = ''.join(filter(str.isdigit, price))
        if price:
            return int(price)
    return None

def clean_mileage(mileage):
    if isinstance(mileage,str):
        mileage = ''.join(filter(str.isdigit, mileage))
        if mileage:
            return int(mileage)
    return None

def extract_year(car):
    match = re.search(r'\b(19|20)\d{2}\b', car)
    if match:
        return int(match.group(0))
    return None




df["Price"] = df["Price"].apply(clean_price)
df["Mileage"] = df["Mileage"].apply(clean_mileage)
df["Seller Rating Count"] = df["Seller Rating Count"].apply(clean_mileage)
df

,Car,Condition,Mileage,Price,Basics Info,Vehicle History Info,Vehicle Reviews Info,Seller Rating,Seller Rating Count,Seller Address
0,2024 Lexus LC 500 Base,New,0.0,1.128650e+05,"{'Exterior color': ' Caviar ', 'Interior color...",{},{},4.7,1261.0,"1250 W Division St Chicago, IL 60642"
1,2007 Acura TSX Base,Used,61110.0,1.129500e+04,{'Exterior color': ' Alabaster Silver Metallic...,{'Accidents or damage': 'At least 1 accident o...,{},4.2,440.0,"1301 N Elston Ave Chicago, IL 60642"
2,2016 McLaren 675LT Base,Used,6305.0,2.199975e+09,"{'Exterior color': ' McLaren Orange ', 'Interi...",{'Accidents or damage': 'At least 1 accident o...,"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",3.1,421.0,"1561 N Fremont St Chicago, IL 60642"
3,2016 Audi TTS 2.0T quattro,Used,65715.0,2.399900e+04,"{'Exterior color': ' Black ', 'Interior color'...",{'Accidents or damage': 'At least 1 accident o...,"{'Comfort': '3.0', 'Interior': '5.0', 'Perform...",3.6,123.0,"560 E North Ave West Chicago, IL 60185"
4,2018 BMW 740e xDrive iPerformance,Used,19830.0,3.979910e+07,"{'Exterior color': ' Imperial Blue Metallic ',...","{'Accidents or damage': 'None reported', 'Clea...","{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.4,91.0,"6539 Ogden Ave Berwyn, IL 60402"
...,...,...,...,...,...,...,...,...,...,...
9241,NaN,NaN,NaN,NaN,{},{},{},NaN,NaN,NaN
9242,2022 BMW X3 xDrive30i,Used,48804.0,2.798000e+07,"{'Exterior color': ' Dark Graphite Metallic ',...","{'Accidents or damage': 'None reported', '1-ow...",{},4.8,3739.0,"1313 Rand Road Des Plaines, IL 60016"
9243,2024 GMC Sierra 1500 Pro,New,3.0,5.108000e+04,"{'Exterior color': ' Summit White ', 'Interior...",{},"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.5,3896.0,"8425 159th St Tinley Park, IL 60487"
9244,2012 GMC Terrain SLT-1,Used,146694.0,8.995000e+03,"{'Exterior color': ' Black ', 'Interior color'...",{'Accidents or damage': 'At least 1 accident o...,{},NaN,NaN,"13840 South Pulaski Road Crestwood, IL 60445"


In [195]:
df_cleaned = df.dropna().copy()
def simplify_condition(condition):
    if 'Certified' in condition:
        return 'Certified'
    return condition


df_cleaned.loc[:, "Condition"] = df_cleaned["Condition"].apply(simplify_condition)
df_cleaned["Year"] = df_cleaned["Car"].apply(extract_year)

# Drop rows with missing year
df_cleaned = df_cleaned.dropna(subset=["Year"])
df_cleaned.loc[:, 'Age'] = 2024 - df_cleaned['Year']
df_cleaned

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

for col in ["Price", "Mileage", "Seller Rating"]:
    df_cleaned = remove_outliers(df_cleaned, col)

df_cleaned


,Car,Condition,Mileage,Price,Basics Info,Vehicle History Info,Vehicle Reviews Info,Seller Rating,Seller Rating Count,Seller Address,Year,Age
0,2024 Lexus LC 500 Base,New,0.0,112865.0,"{'Exterior color': ' Caviar ', 'Interior color...",{},{},4.7,1261.0,"1250 W Division St Chicago, IL 60642",2024,0
1,2007 Acura TSX Base,Used,61110.0,11295.0,{'Exterior color': ' Alabaster Silver Metallic...,{'Accidents or damage': 'At least 1 accident o...,{},4.2,440.0,"1301 N Elston Ave Chicago, IL 60642",2007,17
3,2016 Audi TTS 2.0T quattro,Used,65715.0,23999.0,"{'Exterior color': ' Black ', 'Interior color'...",{'Accidents or damage': 'At least 1 accident o...,"{'Comfort': '3.0', 'Interior': '5.0', 'Perform...",3.6,123.0,"560 E North Ave West Chicago, IL 60185",2016,8
7,2012 Nissan NV Cargo S,Used,110202.0,14985.0,"{'Exterior color': ' Blizzard ', 'Interior col...",{'Accidents or damage': 'At least 1 accident o...,"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",3.6,64.0,"6007 S Archer Rd Summit, IL 60501",2012,12
8,2008 Chrysler Sebring Touring,Used,77893.0,7995.0,"{'Exterior color': ' Silver ', 'Interior color...","{'Accidents or damage': 'None reported', '1-ow...",{},3.5,43.0,"1210 25th Ave Melrose Park, IL 60160",2008,16
...,...,...,...,...,...,...,...,...,...,...,...,...
9234,2024 Jeep Compass Latitude Lux,New,10.0,32990.0,"{'Exterior color': ' Bright White Clearcoat ',...",{},"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.5,1907.0,"4550 W. Lincoln Highway Route 30 Matteson, IL ...",2024,0
9235,2023 Jeep Gladiator Sport S,New,5.0,42995.0,{'Exterior color': ' Granite Crystal Metallic ...,{},"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.5,3345.0,"1155 W Dundee Rd Arlington Heights, IL 60004",2023,1
9236,2024 Subaru Forester Premium,New,6.0,33989.0,"{'Exterior color': ' Autumn Green Metallic ', ...",{},"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.5,221.0,"1911 N Rand Rd Palatine, IL 60074",2024,0
9240,2024 Jeep Grand Cherokee L Limited,New,10.0,52301.0,"{'Exterior color': ' Bright White Clearcoat ',...",{},"{'Comfort': '5.0', 'Interior': '5.0', 'Perform...",4.0,54.0,"4630 95th Street Oak Lawn, IL 60453",2024,0


In [196]:
features = df_cleaned[["Condition", "Mileage", "Seller Rating", "Seller Rating Count", "Age"]]
target = df_cleaned["Price"]

features.loc[:,'Mileage_SellerRating'] = features['Mileage'] * features['Seller Rating']
features.loc[:,'Mileage_SellerRatingCount'] = features['Mileage'] * features['Seller Rating Count']
features.loc[:,'SellerRating_SellerRatingCount'] = features['Seller Rating'] * features['Seller Rating Count']


numerical_features = ["Mileage", "Seller Rating", "Seller Rating Count", "Age"]
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])

categorical_features = ["Condition"]
categorical_transformer = OneHotEncoder(drop="first")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
    
)






pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(objective='reg:squarederror'))
])

param_grid = {
    'regressor__n_estimators': [50, 100, 150],
    'regressor__max_depth': [3, 6, 9],
    'regressor__learning_rate': [0.01, 0.1, 0.3],
    'regressor__subsample': [0.6, 0.8, 1.0],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0],
    'regressor__reg_alpha':  [0.1, 0.5],  # L1 regularization
    'regressor__reg_lambda': [0.1, 0.5]  # L2 regularization
}

scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'R2': make_scorer(r2_score)
}


C:\Users\fayza\AppData\Local\Temp\ipykernel_33056\663310282.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.loc[:,'Mileage_SellerRating'] = features['Mileage'] * features['Seller Rating']
C:\Users\fayza\AppData\Local\Temp\ipykernel_33056\663310282.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.loc[:,'Mileage_SellerRatingCount'] = features['Mileage'] * features['Seller Rating Count']
C:\Users\fayza\AppData\Local\Temp\ipykernel_33056\663310282.py:6: SettingWithCopyWarning: 
A value 

In [197]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=scoring, refit="R2" )

grid_search.fit(X_train, y_train)
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best R2 score: {grid_search.best_score_}')

y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Best parameters: {'regressor__colsample_bytree': 0.6, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 9, 'regressor__n_estimators': 50, 'regressor__reg_alpha': 0.5, 'regressor__reg_lambda': 0.5, 'regressor__subsample': 1.0}
Best R2 score: 0.5504277798152097
Mean Squared Error: 252048171.52743843
R-squared: 0.7120885428621788


In [198]:
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

# Analyze the scale of the target variable
mean_price = np.mean(df_cleaned['Price'])
std_price = np.std(df_cleaned['Price'])

print(f'Mean Price: {mean_price}')
print(f'Standard Deviation of Price: {std_price}')

# Compare RMSE to mean and standard deviation
print(f'RMSE as a percentage of Mean Price: {rmse / mean_price * 100:.2f}%')
print(f'RMSE as a percentage of Standard Deviation: {rmse / std_price * 100:.2f}%')

Root Mean Squared Error: 15876.025054384314
Mean Price: 43553.04119547657
Standard Deviation of Price: 29433.903212688678
RMSE as a percentage of Mean Price: 36.45%
RMSE as a percentage of Standard Deviation: 53.94%


In [199]:
joblib.dump(grid_search.best_estimator_, 'best_model_pipeline.pkl')


['best_model_pipeline.pkl']